# Chapter 13: Human-in-the-Loop


Key Takeaways:
- **Human-Agent Collaboration**: Design systems where agents and humans work together, handling different levels of complexity.
- **Escalation Patterns**: Implement clear escalation paths for agents to hand off tasks to humans when they exceed their capabilities.
- **Contextual Awareness**: Agents should leverage user context and history to provide personalized and relevant support.
- **Tool-Based Actions**: Use tools to perform actions like troubleshooting, ticketing, and escalation.

### Heuristic: *Design for escalation. Let agents handle the routine, and humans handle the exceptions.*

## Setup and Initialization

In [ ]:
import os
import sys
from dotenv import load_dotenv
from langchain_core.tools import tool
from typing import Optional, Dict, List, Callable, Any
from langchain_core.messages import SystemMessage

# Add scripts directory to path to import custom modules
PROJECT_ROOT = os.path.dirname(os.getcwd())
SCRIPTS_DIR = os.path.join(PROJECT_ROOT, "scripts")
sys.path.insert(0, SCRIPTS_DIR)

# Import our custom ADK implementation
from custom_adk import Agent

# Load environment variables
load_dotenv()

# Verify API key
if not os.getenv("GOOGLE_API_KEY"):
    print("❌ Please set the GOOGLE_API_KEY environment variable.")
else:
    print("✅ Configuration Loaded")

## 1. Define Tools

We will define tools for troubleshooting, ticket creation, and human escalation.

In [ ]:
@tool
def troubleshoot_issue(issue: str) -> Dict[str, str]:
    """Provides troubleshooting steps for a reported issue."""
    return {"status": "success", "report": f"Troubleshooting steps for {issue}."}

@tool
def create_ticket(issue_type: str, details: str) -> Dict[str, str]:
    """Creates a support ticket for tracking."""
    return {"status": "success", "ticket_id": "TICKET123"}

@tool
def escalate_to_human(issue_type: str) -> Dict[str, str]:
    """Escalates the issue to a human specialist."""
    # This would typically transfer to a human queue in a real system
    return {"status": "success", "message": f"Escalated {issue_type} to a human specialist."}

## 2. Define Callback

We verify customer history and inject personalization into the agent's context using a callback.

In [ ]:
def personalization_callback(agent: Agent, messages: List[Any], state: Dict[str, Any]) -> None:
    """Adds personalization information to the LLM request based on state."""
    # Get customer info from state
    customer_info = state.get("customer_info")
    if customer_info:
        customer_name = customer_info.get("name", "valued customer")
        customer_tier = customer_info.get("tier", "standard")
        recent_purchases = customer_info.get("recent_purchases", [])
        
        personalization_note = (
            f"\nIMPORTANT PERSONALIZATION:\n"
            f"Customer Name: {customer_name}\n"
            f"Customer Tier: {customer_tier}\n"
        )
        if recent_purchases:
            personalization_note += f"Recent Purchases: {', '.join(recent_purchases)}\n"
        
        # Add as a system message before the first content (which is the instruction)
        # We insert at index 1 because index 0 is the main instruction SystemMessage
        system_content = SystemMessage(content=personalization_note)
        messages.insert(1, system_content)

## 3. Define Agent

We create the Technical Support Specialist agent.

In [ ]:
technical_support_agent = Agent(
    name="technical_support_specialist",
    model="gemini-2.5-flash",
    instruction="""
You are a technical support specialist for our electronics company.
FIRST, check if the user has a support history in state["customer_info"]["support_history"]. If they do, reference this history in your responses.

For technical issues:
1. Use the troubleshoot_issue tool to analyze the problem.
2. Guide the user through basic troubleshooting steps.
3. If the issue persists, use create_ticket to log the issue.

For complex issues beyond basic troubleshooting:
1. Use escalate_to_human to transfer to a human specialist.

Maintain a professional but empathetic tone. Acknowledge the frustration technical issues can cause, while providing clear steps toward resolution.
""",
    tools=[troubleshoot_issue, create_ticket, escalate_to_human],
    callbacks=[personalization_callback]
)

## 4. Execution Examples

### Scenario 1: Automated Resolution
A standard issue that the agent can handle with troubleshooting.

In [ ]:
print("🟢 --- SCENARIO 1: AUTOMATED RESOLUTION ---")
state_1 = {
    "last_user_message": "My screen is flickering.",
    "customer_info": {
        "name": "Alice",
        "tier": "Gold",
        "recent_purchases": ["Monitor X1000"],
        "support_history": []
    }
}

response_1 = technical_support_agent.run(state_1)

### Scenario 2: Human Escalation
A complex issue that requires human intervention.

In [ ]:
print("\n🔴 --- SCENARIO 2: HUMAN ESCALATION ---")
state_2 = {
    "last_user_message": "I've tried everything, smoke is coming out of the device!",
    "customer_info": {
        "name": "Bob",
        "tier": "Platinum",
        "recent_purchases": ["Server Rack Pro"],
        "support_history": ["Previous overheating issue"]
    }
}

response_2 = technical_support_agent.run(state_2)

## Conclusion

This pattern demonstrates how to integrate human-in-the-loop workflows into agentic systems. By using callbacks for context injection and specific tools for escalation, we ensure that agents provide personalized service and gracefully hand off to humans when necessary.